In [231]:
import pandas as pd
import numpy as np
import scipy as sp
import pulp
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, LpMinimize

In [232]:
# Assuming FoodDatabase.csv is in the same directory as your Jupyter Notebook
file_path = 'FoodDatabase.csv'
#All numbers (macros and micros) in the table are a serving of 100g
#When adding new foods to the csv, must fill in: protein / kg column, protein quality column (1 or 0), and product name column.

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Filter rows with nonzero 'Price / kg'
df_filtered = df[df['Price / kg'] != 0]

# Replace NaNs with 0s
df_filtered = df_filtered.fillna(0)

# Add 'Cost ($)' column with initial value of zero at the start
df_filtered.insert(0, 'Cost ($)', 0)

# Add Quantities' (g)' column with initial value of zero as the second column
df_filtered.insert(1, 'Quantities (g)', 0)

# Display the first few rows of the DataFrame to verify it was read correctly
df_filtered.info()
df_filtered.head()


#We want to minimize c_1 * x_1 + c_2 * x_2 ... c_n * x_n , where c is the cost per unit mass of a given food, and x is the mass of the given food (aka the objective function)

# The constraints are the macros and micros of a diet, and other conditions we set and must be met. 

#Excel was using Simplex LP method


<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 751 to 14169
Columns: 107 entries, Cost ($) to Pills?
dtypes: float64(98), int64(3), object(6)
memory usage: 23.6+ KB


,Cost ($),Quantities (g),ID,name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),...,Theobromine (mg),200 Calorie Weight (g),Sugar (g),Price / kg,Protein Quality,Product Name,Last Updated,Source,Comments,Pills?
751,0,0,168263,Pork Fresh Loin Center Loin (Chops) Boneless S...,Meats,122.81,3.09,23.75,0.00,0.00,...,0.0,162.602,0.00,5.158811,1.0,Smithfield Fresh Pork Center Cut Loin Boneless...,01-Feb-24,https://www.walmart.com/ip/Smithfield-Fresh-Po...,0,0.0
1365,0,0,168877,Rice White Long-Grain Regular Raw Enriched,Grains and Pasta,349.06,0.66,7.13,79.95,0.12,...,0.0,54.795,0.06,1.228000,0.0,"Great Value Long Grain Enriched Rice, 20 lb USA",01-Feb-24,https://www.walmart.com/ip/Great-Value-Lentils...,0,0.0
1382,0,0,168894,Wheat Flour White All-Purpose Enriched Bleached,Grains and Pasta,344.58,0.98,10.33,76.31,0.27,...,0.0,54.945,0.00,1.080264,0.0,"Great Value All-Purpose Flour, 5LB Bag USA",24-Feb-24,https://www.walmart.com/ip/Great-Value-All-Pur...,0,0.0
1775,0,0,169287,Spinach Frozen Chopped Or Leaf Unprepared,Vegetables,24.89,0.57,3.63,4.21,0.65,...,0.0,689.655,0.30,3.410000,0.0,"Great Value Chopped Spinach, 12 oz (Frozen)",0,0,0,0.0
2224,0,0,169736,Pasta Dry Enriched,Grains and Pasta,351.63,1.51,13.04,74.67,2.67,...,0.0,53.908,0.35,1.999590,0.0,"Great Value Elbows, 48 oz PASTA USA",01-Feb-24,https://www.walmart.com/ip/Great-Value-Elbows-...,"not worth it in the US, dairy better, pricey",0.0


In [233]:
from pulp import LpProblem, lpSum

# Define base class for constraints
class Constraint:
    def __init__(self, model, variables, coefficients, name):
        self.model = model
        self.variables = variables
        self.coefficients = coefficients
        self.name = name

    def get_sum(self):
        return lpSum(self.variables[i] * self.coefficients[i-1] for i in range(1, len(self.variables) + 1))

class RangeConstraint(Constraint):
    def __init__(self, model, variables, coefficients, name, lower_bound, upper_bound):
        super().__init__(model, variables, coefficients, name)
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound

    def add_constraints(self):
        sum_expr = self.get_sum()
        self.model += (sum_expr >= self.lower_bound, f"{self.name} Lower Bound Constraint")
        self.model += (sum_expr <= self.upper_bound, f"{self.name} Upper Bound Constraint")

class MinimumConstraint(Constraint):
    def __init__(self, model, variables, coefficients, name, minimum):
        super().__init__(model, variables, coefficients, name)
        self.minimum = minimum

    def add_constraint(self):
        sum_expr = self.get_sum()
        self.model += (sum_expr >= self.minimum, f"{self.name} Minimum Constraint")

class EqualityConstraint(Constraint):
    def __init__(self, model, variables, coefficients, name, value):
        super().__init__(model, variables, coefficients, name)
        self.value = value

    def add_constraint(self):
        sum_expr = self.get_sum()
        self.model += (sum_expr == self.value, f"{self.name} Equality Constraint")

class MaximumConstraint(Constraint):
    def __init__(self, model, variables, coefficients, name, maximum):
        super().__init__(model, variables, coefficients, name)
        self.maximum = maximum

    def add_constraint(self):
        sum_expr = self.get_sum()
        self.model += (sum_expr <= self.maximum, f"{self.name} Maximum Constraint")

# Define the model (for maintenance macro + micros)
maintenance_model = LpProblem(name="Maintenance_Diet_Minimum_Cost", sense=LpMinimize)

# Define the decision variables (what we are solving for; quantities of each food)
x = {i: LpVariable(name=f"x_{i}", lowBound=0, upBound=10000) for i in range(1, len(df_filtered)+1)}

# Function to extract column array from a DataFrame with optional scaling
def extract_column_array(df, column, scale_factor=1):
    values = df.iloc[:, column].values
    if np.issubdtype(values.dtype, np.number):
        return values / scale_factor
    return values

# Index dictionary for human-readable names
index = {
    "dollars_per_g": (100, 1000),
    "name_per_food": (102, 1),
    "category_per_food": (4, 1),
    "is_pill_per_food": (106, 1), # is or is not a pill (supplement)
    "calories_per_g": (5, 100), # scale factor in these is 100 to convert 100g into g
    "protein_per_g": (7, 100),
    "net_carbs_per_g": (95, 100),
    "fiber_per_g": (10, 100),
    "fat_per_g": (6, 100),
    "protein_quality": (101, 1),
    "sugar_per_g": (99, 100),
    "satfat_per_g": (12, 100),
    "calcium_per_g": (13, 100),
    "iron_per_g": (14, 100),
    "potassium_per_g": (15, 100),
    "magnesium_per_g": (16, 100),
    "sodium_per_g": (38, 100),
    "zinc_per_g": (39, 100),
    "copper_per_g": (40, 100),
    "manganese_per_g": (41, 100),
    "selenium_per_g": (42, 100),
    "vitaminC_per_g": (19, 100),
    "niacin_per_g": (48, 100),
    "EPA_per_g": (72, 100),
    "DHA_per_g": (74, 100),
}

# Function to get the array for a given nutrient type
def get_nutrient_array(nutrient_type: str):
    if nutrient_type in index:
        col, scale = index[nutrient_type]
        return extract_column_array(df_filtered, col, scale)
    else:
        raise ValueError(f"Nutrient type '{nutrient_type}' is not recognized")

dollars_per_g = get_nutrient_array("dollars_per_g")
name_per_food = get_nutrient_array("name_per_food")
category_per_food = get_nutrient_array("category_per_food")
is_pill_per_food = get_nutrient_array("is_pill_per_food")

# Macros arrays
calories_per_g = get_nutrient_array("calories_per_g")
protein_per_g = get_nutrient_array("protein_per_g")
net_carbs_per_g = get_nutrient_array("net_carbs_per_g")
fiber_per_g = get_nutrient_array("fiber_per_g")
fat_per_g = get_nutrient_array("fat_per_g")
protein_quality = get_nutrient_array("protein_quality")
sugar_per_g = get_nutrient_array("sugar_per_g")
satfat_per_g = get_nutrient_array("satfat_per_g")

# Micros arrays
calcium_per_g = get_nutrient_array("calcium_per_g")
iron_per_g = get_nutrient_array("iron_per_g")
potassium_per_g = get_nutrient_array("potassium_per_g")
magnesium_per_g = get_nutrient_array("magnesium_per_g")
sodium_per_g = get_nutrient_array("sodium_per_g")
zinc_per_g = get_nutrient_array("zinc_per_g")
copper_per_g = get_nutrient_array("copper_per_g")
manganese_per_g = get_nutrient_array("manganese_per_g")
selenium_per_g = get_nutrient_array("selenium_per_g")
vitaminC_per_g = get_nutrient_array("vitaminC_per_g")
niacin_per_g = get_nutrient_array("niacin_per_g")
DHA_per_g = get_nutrient_array("EPA_per_g")
EPA_per_g = get_nutrient_array("DHA_per_g")
FishOils_per_g = EPA_per_g + DHA_per_g


# Objective Function
maintenance_model += lpSum(x[i] * dollars_per_g[i-1] for i in range(1, len(df_filtered)+1))

# Defining Constraints
# TODO: PULL THESE FROM SOMEWHERE ELSE, MAYBE READ CSV THAT HAS BODYWEIGHT AND OTHER STATS AND COMPUTE THE MAINTENANCE AND OTHER NEEDS BASED ON 
# Macros
maintenance_calories = 2937  # set maintenance calories
protein_minimum = 242  # set protein minimum
net_carb_minimum = 236  # set net carb minimum
fiber_lbound = 30  # lower bound for fiber
fiber_ubound = 60  # upper bound for fiber
fat_minimum = 82
protein_quality_minimum = 0.4  # this is a fraction ie 0.4 = 40%
sugar_maximum = 75
sat_fat_maximum = 0.5  # this is a fraction

# Macro constraints using classes
macro_constraints = [
    EqualityConstraint(maintenance_model, x, calories_per_g, "Maintenance Calories", maintenance_calories),
    MinimumConstraint(maintenance_model, x, protein_per_g, "Protein", protein_minimum),
    MinimumConstraint(maintenance_model, x, net_carbs_per_g, "Net Carb", net_carb_minimum),
    RangeConstraint(maintenance_model, x, fiber_per_g, "Fiber", fiber_lbound, fiber_ubound),
    MinimumConstraint(maintenance_model, x, fat_per_g, "Fat", fat_minimum),
    MinimumConstraint(maintenance_model, x, [protein_quality[i-1] * protein_per_g[i-1] for i in range(1, len(x)+1)], "Protein Quality", protein_quality_minimum * lpSum(x[i] * protein_per_g[i-1] for i in range(1, len(x)+1))), # Remember, divisions need to be reformulated in the inequality as multiplications to solve properly.
    MaximumConstraint(maintenance_model, x, sugar_per_g, "Sugar", sugar_maximum),
    MaximumConstraint(maintenance_model, x, [satfat_per_g[i-1] for i in range(1, len(x)+1)], "SatFat", sat_fat_maximum * lpSum(x[i] * fat_per_g[i-1] for i in range(1, len(x)+1))) # Currently Fast and loose with fat rules. Sat fats no more than 50% of total fats. May change this in the future.
]

# Add macro constraints to the model
for constraint in macro_constraints:
    constraint.add_constraint() if isinstance(constraint, (MinimumConstraint, EqualityConstraint, MaximumConstraint)) else constraint.add_constraints()

# Micros (mg unless otherwise stated)
calcium_lbound = 1500
calcium_ubound = 2000
iron_lbound = 30
iron_ubound = 40
potassium_maximum = 10000
magnesium_lbound = 600  # Maybe play around with the calcium to magnesium ratios; apparently should be within 1.70 - 2.60 , but high calcium recommendations and maximum magnesium daily intake of 420 mg makes that difficult to attain. Let's stretch it to 1.70 - 3.15 (this probably doesn't matter if you get enough of everything)
magnesium_ubound = 700
sodium_maximum = 5000  # because potassium maximum is 10k, you want potassium to sodium ratio to be 2:1 or higher; but if you sweat a lot, probably can exceed it. just make sure potassium is at least 1:1 with sodium.
zinc_lbound = 35
zinc_ubound = 40
copper_maximum = 10
manganese_lbound = 8 # Probably more than ample
manganese_ubound = 20
selenium_maximum = 350  # micrograms
vitaminC_lbound = 120 # spread between 3 meals? 
vitaminC_ubound = 180
niacin_minimum = 25
Fish_Oils_minimum = 3000 # we want 3g EPA+DHA daily

# Define constraints
micro_constraints = [
    RangeConstraint(maintenance_model, x, calcium_per_g, "Calcium", calcium_lbound, calcium_ubound),
    RangeConstraint(maintenance_model, x, iron_per_g, "Iron", iron_lbound, iron_ubound),
    MaximumConstraint(maintenance_model, x, potassium_per_g, "Potassium", potassium_maximum),
    RangeConstraint(maintenance_model, x, magnesium_per_g, "Magnesium", magnesium_lbound, magnesium_ubound),
    MaximumConstraint(maintenance_model, x, sodium_per_g, "Sodium", sodium_maximum),
    RangeConstraint(maintenance_model, x, zinc_per_g, "Zinc", zinc_lbound, zinc_ubound),
    MaximumConstraint(maintenance_model, x, copper_per_g, "Copper", copper_maximum),
    RangeConstraint(maintenance_model, x, manganese_per_g, "Manganese", manganese_lbound, manganese_ubound),
    MaximumConstraint(maintenance_model, x, selenium_per_g, "Selenium", selenium_maximum),
    RangeConstraint(maintenance_model, x, vitaminC_per_g, "Vitamin C", vitaminC_lbound, vitaminC_ubound),
    MinimumConstraint(maintenance_model, x, niacin_per_g, "Niacin", niacin_minimum),
    MinimumConstraint(maintenance_model, x, FishOils_per_g, "Fish Oils", Fish_Oils_minimum),
]

# Add constraints to the model
for constraint in micro_constraints:
    constraint.add_constraints() if isinstance(constraint, RangeConstraint) else constraint.add_constraint()

# Display the full model info (comment out when not needed)
#print(maintenance_model)

In [234]:
# Solve the optimization problem
status = maintenance_model.solve()

# Get the results and format them
print(f"status: {maintenance_model.status}, {LpStatus[maintenance_model.status]}")
monthly_cost = round(maintenance_model.objective.value() * 30, 2)
objective_value = round(maintenance_model.objective.value(), 2)
print(f"TOTAL COST OF DAILY DIET CALCULATED:\t${objective_value}")
print(f"TOTAL COST OF MONTHLY DIET CALCULATED:\t${monthly_cost}")


# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Food Name', 'Quantity (g)', 'Category', 'is Pill'])

# Collect the data for the DataFrame
results_list = [{'Food Name': name_per_food[i-1], 'Category': category_per_food[i-1],'is Pill': is_pill_per_food[i-1], 'Quantity (g)': int(round(var.value(), 0))} for i, var in x.items() if var.value() > 0]

# Convert the list to a DataFrame and sort
results_df = pd.DataFrame(results_list).sort_values(by='Quantity (g)', ascending=False)

# Move 'Supplements' category to the end
supplements_df = results_df[results_df['Category'] == 'Supplements']
other_categories_df = results_df[results_df['Category'] != 'Supplements']

# Concatenate both parts, ensuring 'Supplements' is at the end``
results_df = pd.concat([other_categories_df, supplements_df])


# Display the results DataFrame
print("\nCONSISTING OF THE FOLLOWING FOODS:")
for index, row in results_df.iterrows():
    food_name = row['Food Name']
    if row['is Pill'] == 1:
        quantity = f"{int(round(row['Quantity (g)'] / 100, 0))} caps"
    else:
        quantity = f"{int(row['Quantity (g)'])} g"
    category = row['Category']
    print(f"{food_name.ljust(95)}: {quantity.ljust(20)}{category}")




# Print errors per constraint equation
print("\nERROR PER CONSTRAINT EQUATION:")
for name, constraint in maintenance_model.constraints.items():
    # Replace underscores with spaces
    name_with_spaces = name.replace('_', ' ')

    # Determine the operator and RHS value
    if "Lower" in name:
        operator = '>='
        rhs_value = -constraint.constant
    elif "Upper" in name:
        operator = '<='
        rhs_value = -constraint.constant
    elif "Minimum" in name:
        operator = '>='
        rhs_value = -constraint.constant
    elif "Equality" in name:
        operator = '=='
        rhs_value = -constraint.constant
    elif "Maximum" in name:
        operator = '<='
        rhs_value = -constraint.constant

    # Calculate the left-hand side value
    lhs_value = sum(var.value() * coeff for var, coeff in constraint.items())

    # Round the lhs_value and rhs_value to the nearest integer
    lhs_value_rounded = round(lhs_value)
    rhs_value_rounded = round(rhs_value)

    # Calculate the error and round to three significant figures
    constraint_value_rounded = round(abs(lhs_value - rhs_value), 2)

    # Format the output
    if constraint_value_rounded >= 0.1:
        print(f"{name_with_spaces.ljust(55)}\t{lhs_value_rounded}\t{operator}\t{rhs_value_rounded},\t{constraint_value_rounded}")
    else:
        print(f"{name_with_spaces.ljust(55)}\t{lhs_value_rounded}\t{operator}\t{rhs_value_rounded}")



status: 1, Optimal
TOTAL COST OF DAILY DIET CALCULATED:	$4.29
TOTAL COST OF MONTHLY DIET CALCULATED:	$128.62

CONSISTING OF THE FOLLOWING FOODS:
Perdue, Fresh Chicken Gizzards (May Contain Hearts), 1.25 lb. Cup USA                          : 418 g               Meats
Great Value Lentils, 1 lb USA                                                                  : 299 g               Beans and Lentils
Perdue, No Antibiotics Ever, Fresh Chicken Leg Quarters, 10 lb. Bag USA                        : 274 g               Meats
Great Value Reduced-Fat Singles American Pasteurized Process Cheese Food, 16 oz, 24 Count USA  : 192 g               Dairy and Egg Products
Popcorn Kernels USA 2 lb                                                                       : 135 g               Snacks
Great Value Frozen Broccoli Cuts, 32 oz Steamable  Raw USA                                     : 96 g                Vegetables
Mom's Best Cereals Whole Grain Oatmeal, Quick Oats, 16 Oz USA                     

In [235]:
#combined = EPA_per_g + DHA_per_g
#print(DHA_per_g)
#print(EPA_per_g)
#rint(category_per_food) 

In [236]:
#Debugger cell
#protein_quality = df_filtered.iloc[:, 101].values

print (len(protein_quality))
print(is_pill_per_food)


28
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1.]


# Thoughts

Iodine data is missing. not a big deal, there is a USDA database where they measured iodine amounts for 467 foods, but the ID columns dont match, I could do somesort of index match fuzzy search, but iodine isn't that important. dont have much recommendations other than don't exceed 600 mcg.